In [ ]:
import polars as pl
import numpy as np

In [ ]:
df = pl.read_parquet("./data/commodities.parquet")
# df_pd = df.to_pandas()

In [ ]:
def query_similarity_count(x:pl.Expr, subseq:list[float], threshold:float, z_normalize:bool=False):
    '''
    '''
    # If we want to allow other distances metrics, one way 
    # is to change pl.element().pow(2).sum().sqrt() step

    s = pl.Series(subseq, dtype=pl.Float64)
    if z_normalize:
        s = (s - s.mean())/s.std()
        mean = pl.concat_list(x, *(x.shift(-i) for i in range(1, len(s)))).list.eval(pl.element().mean()).list.first()
        std = pl.concat_list(x, *(x.shift(-i) for i in range(1, len(s)))).list.eval(pl.element().std()).list.first()
        expr = (
            pl.concat_list((x-mean)/std - pl.lit(s[0]), *(((x.shift(-i)-mean)/std - pl.lit(s[i])) 
                                                          for i in range(1, len(s))))
            .list.eval(pl.element().pow(2).sum().sqrt())
            .list.first().lt(threshold).sum()
        )

    else:
        expr = (
            pl.concat_list(x - s[0], *(x.shift(-i) - s[i] for i in range(1, len(s))))
            .list.eval(pl.element().pow(2).sum().sqrt())
            .list.first().lt(threshold).sum()
        )

    return expr

In [ ]:
subseq = [511.47, 511.47, 511.47]

In [ ]:
x = pl.col("price")
subseq = [511.47, 511.47, 511.47]
s = pl.Series(subseq)
df.select(
    query_similarity_count(x, subseq, 10, z_normalize=True)
    # z_normalized 
)

In [ ]:
import stumpy

q = df["price"].head(3).to_numpy()
t = df["price"].to_numpy()
stumpy.core.mass(q,t)

In [ ]:
q = np.array([1,2,3]).astype(np.float64)
t = np.array(range(1,101)).astype(np.float64)
stumpy.core.mass(q,t)

In [ ]:
from functime.feature_extraction.tq_feature_extractions import permutation_entropy
import tsfresh.feature_extraction.feature_calculators as fc

In [ ]:

fc.permutation_entropy(df["price"], tau = 1, dimension=3)

In [ ]:

df.lazy().select(
    permutation_entropy(pl.col("price"), tau=1, n_dims=3)
).collect().item(0,0)